# Tutorial 4: Memory- and time-efficient solving of ME-models

In this tutorial we will convert the ME-model object to an NLP mathematical representation to save memory and time in simulating many conditions.

## Import libraries

In [1]:
import coralme
from helpers import get_nlp,optimize

## Load

Load the ME-model coming out of the Troubleshooter

In [2]:
me = coralme.io.json.load_json_me_model("MEModel-step3-bsubtilis-TS.json")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16


Adding Metabolites into the ME-model...                                    : 100.0%|██████████|  4432/ 4432 [00:00<00:00]
Adding ProcessData into the ME-model...                                    : 100.0%|██████████|  4484/ 4484 [00:00<00:00]
Adding Reactions into the ME-model...                                      : 100.0%|██████████|  7461/ 7461 [00:16<00:00]
Updating ME-model Reactions...                                             : 100.0%|██████████|  6068/ 6068 [00:16<00:00]


## Convert to NLP problem

The ME-model object *me* is a big object containing all data and metadata. This is not necessary when performing large-scale simulations, such as gene knockouts, or growth simulations under hundreds of conditions.

So, in these cases we only need the mathematical problem representing the ME-model, which is *nlp*.

In [3]:
nlp = get_nlp(me)

## Retrieve metabolite and reaction indexes

The *nlp* now contains the mathematical representation, very similar to a struct object of the COBRA Toolbox in MATLAB. Similarly, reactions and metabolites are now accessed from integer indexes. We can create a dictionary from the original model to map reaction ids to indexes

In [4]:
rxn_index_dct = {r.id : me.reactions.index(r) for r in me.reactions}
met_index_dct = {m.id : me.metabolites.index(m) for m in me.metabolites}

From now on, *me* is no longer necessary and can be deleted to save memory usage. This is especially helpful when running parallelized simulations.

In [5]:
# del me

## Solving the MEModel vs. NLP

Now we can call the modified *optimize* function in *helpers*. This function was modified from the me.optimize() function of a coralme.core.model.MEModel.

Here you can see the speed-up when solving from scratch and solving from the NLP. The speed-up is even more noticeable with bigger models, as lamdifying a longer list of constraints will take much longer.

### ME-model

In [6]:
%%time
me.optimize(max_mu = 1.0, min_mu = 0., maxIter = 100, lambdify = True,
		tolerance = 1e-6, precision = 'quad', verbose = True)

The MINOS and quad MINOS solvers are a courtesy of Prof Michael A. Saunders. Please cite Ma, D., Yang, L., Fleming, R. et al. Reliable and efficient solution of genome-scale models of Metabolism and macromolecular Expression. Sci Rep 7, 40863 (2017). https://doi.org/10.1038/srep40863

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Not feasible
        2	0.2500000000000000	Not feasible
        3	0.1250000000000000	Not feasible
        4	0.0625000000000000	Optimal
        5	0.0937500000000000	Not feasible
        6	0.0781250000000000	Optimal
        7	0.0859375000000000	Optimal
        8	0.0898437500000000	Optimal
        9	0.0917968750000000	Optimal
       10	0.0927734375000000	Not feasible
       11	0.0922851562500000	Not feasible
       12	0.0920410156250000	Not feasible
       13	0.0919189453125000	Not feasible
       14	0.0918579101562500	Optimal
       15	0.0918884277343750	Optimal
       16	0.0919036865234375	Opti

True

### NLP

In [7]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 1.0, min_mu = 0., maxIter = 100,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Not feasible
        2	0.2500000000000000	Not feasible
        3	0.1250000000000000	Not feasible
        4	0.0625000000000000	Optimal
        5	0.0937500000000000	Not feasible
        6	0.0781250000000000	Optimal
        7	0.0859375000000000	Optimal
        8	0.0898437500000000	Optimal
        9	0.0917968750000000	Optimal
       10	0.0927734375000000	Not feasible
       11	0.0922851562500000	Not feasible
       12	0.0920410156250000	Not feasible
       13	0.0919189453125000	Not feasible
       14	0.0918579101562500	Optimal
       15	0.0918884277343750	Optimal
       16	0.0919036865234375	Optimal
       17	0.0919113159179688	Not feasible
       18	0.0919075012207031	Optimal
       19	0.0919094085693359	Not feasible
       20	0.0919084548950195	Optimal
CPU times: user 2min 31s, sys: 9.1 ms, total: 2min 31s
Wall time: 2min 31s


## Re-using the basis for even more speed-up

We can re-use a basis from a previously successful simulation to warm-start the first iteration and save even more time! 

### Re-using basis

In [9]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.2, min_mu = 0., maxIter = 1,
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.1000000000000000	Not feasible
CPU times: user 4.28 s, sys: 0 ns, total: 4.28 s
Wall time: 4.27 s


### Cold start

In [8]:
%%time
sol,_ = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 0.2, min_mu = 0., maxIter = 1, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = None)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.1000000000000000	Not feasible
CPU times: user 52.6 s, sys: 828 µs, total: 52.6 s
Wall time: 52.4 s


**Speed-up from ~50 seconds to ~4 seconds!**

## Modifying the NLP

As previously mentioned, the NLP resembles a struct object of the COBRA Toolbox. The model is stored as a collection of vectors and matrices representing stoichiometries, bounds and other variables needed by the solvers.

The relevant properties are:
* **xu**: Upper bounds
* **xl**: Lower bounds
* **S**: Stoichiometric matrix (Metabolites x Reactions)

The carbon source right now is Glucose, so we will change its bound to -10 to try to achieve maximum growth rate. 

**Note that bounds contain *lambdify* objects, not floats!**

In [10]:
nlp.xl[rxn_index_dct["EX_glc__D_e"]] = lambda x:-10

In [11]:
%%time
sol,basis = optimize(rxn_index_dct,met_index_dct,nlp,max_mu = 1.0, min_mu = 0., maxIter = 100, 
		tolerance = 1e-6, precision = 'quad', verbose = True, basis = basis)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Not feasible
        2	0.2500000000000000	Optimal
        3	0.3750000000000000	Optimal
        4	0.4375000000000000	Not feasible
        5	0.4062500000000000	Optimal
        6	0.4218750000000000	Optimal
        7	0.4296875000000000	Not feasible
        8	0.4257812500000000	Optimal
        9	0.4277343750000000	Optimal
       10	0.4287109375000000	Optimal
       11	0.4291992187500000	Not feasible
       12	0.4289550781250000	Optimal
       13	0.4290771484375000	Optimal
       14	0.4291381835937500	Not feasible
       15	0.4291076660156250	Optimal
       16	0.4291229248046875	Optimal
       17	0.4291305541992188	Not feasible
       18	0.4291267395019531	Optimal
       19	0.4291286468505859	Not feasible
       20	0.4291276931762695	Not feasible
CPU times: user 1min 50s, sys: 459 µs, total: 1min 50s
Wall time: 1min 50s


## Inspecting the solution

The function returns a cobra.Solution object just like the one stored in me.solution. For more details inspecting *sol*, refer to Tutorial 3.

In [12]:
sol

,fluxes,reduced_costs
biomass_dilution,4.291267e-01,-4.399842e-01
BSU00360-MONOMER_to_generic_16Sm4Cm1402,6.702834e-11,0.000000e+00
BSU15140-MONOMER_to_generic_16Sm4Cm1402,0.000000e+00,-5.352912e+00
RNA_BSU_rRNA_1_to_generic_16s_rRNAs,0.000000e+00,-2.975286e-34
RNA_BSU_rRNA_16_to_generic_16s_rRNAs,0.000000e+00,-2.248306e-01
...,...,...
TS_pydx5p_c,-1.139554e-06,0.000000e+00
TS_zn2_c,-1.477569e-06,0.000000e+00
TS_fe2_c,-2.334034e-08,0.000000e+00
TS_thmpp_c,-3.360880e-07,0.000000e+00
